```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```


# BERT: Fine-tunning para modelos de clasificación

En este notebook, intentaremos resolver el problema de clasificación de tweets con el que venimos trabajando pero aplicando un modelo basado en la arquitectura de BERT y entrenado para la tarea especifica que queremos resolver utilizando el concepto de Transfer Learning.

## Preparación del ambiente

In [25]:
import warnings
warnings.filterwarnings('ignore')

Descarguemos algunos fragmentos de código para simplificar el trabajo

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/ClassificationDataset.py --directory-prefix ./Utils/

Descargamos el set de datos

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

Instalamos las librerias necesarias

In [ ]:
!pip install transformers

Cargamos el set de datos

In [3]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

## Sobre Fine-Tunning

En general, existen 2 estrategias para utilizar modelos de lenguaje pre-entrenados en una tarea especifica:
 - Feature-based
 - Fine-tunning
 
Las técnicas que se conocen como **Feature-based** utiliza arquitecturas especificas para resolver cada una de las tareas de NLP, en donde los pesos de las representaciones vectoriales están "congeladas" y no son parámetros que el modelo deba optimizar. En consecuencia, estos modelos son más rápidos de entrenar y permiten aplicar arquitecturas especificas que sean diferenciales en cada una de las tareas.
 
Por el otro lado, las técnicas que emplean **Fine-tunning** tiene la flexibilidad de poder adaptar sus representaciones al permitir que todos los parametros sean optimizados en la tarea en particular. Además, estas arquitecturas permiten resolver multiples problemas de NLP utilizando una mínima cantidad de parametros específicos para la tarea.

## BETO: BERT en español

Al igual que con word2vec, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días).Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de texto. Pueden encontrar más información sobre el autor de este modelo en: https://github.com/dccuchile/beto 

### Tokenizers 

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Recuerden que los tokenizers dependen del modelo con el que estamos trabajando:

In [5]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', do_lower_case=True)

*Noten que el tokenizer depende del modelo que estamos utilizando*

## Crando un modelo de clasificación basado en BERT

Trataremos de resolver entonces el mismo problema de clasificación con el que veniamos trabajando: clasificar los tweets dependiendo del sector al que pertenecen.Recordemos que tenemos 7 categorias distintas:

In [7]:
tweets['SECTOR'].unique()

array(['RETAIL', 'TELCO', 'ALIMENTACION', 'AUTOMOCION', 'BANCA',
       'BEBIDAS', 'DEPORTES'], dtype=object)

Necesitaremos contar con el numero de categorias para nuestro clasificador:

In [8]:
num_labels=len(tweets['SECTOR'].unique())

Antes de hacer fine-tunning de nuestro modelo, tenemos que instanciar el modelo sobre el cual queremos aplicar esta técnica. Para ello instanciaremos el modelo base el cual no está entrenado en ninguna tarea en particular. De hecho, si habilitan las alertas en este notebook, verán que cuando se carga el modelo, la libreria HuggingFace les advierte sobre esto:

In [9]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', num_labels=num_labels)

Construiremos nuestro dataset sobre el que queremos entrenar el modelo. Recuerden que ya habíamos separado el set de datos en porciones para entrenar y para testear el modelo.

### Como entrenar modelos con Transformers

La librería transformers puede entrenar modelos tanto utilizando TensorFlow como PyTorch como backend. En nuestro caso utilizaremos PyTorch simplemente porque generaremos código un poco más compacto, pero pueden utilizar el backend con el que más cómodos se sientan:

### Set de datos

Para utilizar el objeto Trainer que provee transformers, necesitamos crear un objetivo de tipo Dataset. PyTorch implementa este objeto el modulo torch.utils.data.Dataset. Para simplificar la tarea, les he creado una clase que hace todo el procesamiento de datos y generación de los sets de datos utilizando dicho modulo. Pueden encontrar esta implementación en Utils.ClassificationDataset:

In [10]:
from Utils.ClassificationDataset import ClassificationDataset

train_dataset = ClassificationDataset(examples=X_train, labels=y_train, tokenizer=tokenizer)
val_dataset = ClassificationDataset(examples=X_test, labels=y_test, tokenizer=tokenizer)

In [14]:
from transformers import Trainer, TrainingArguments

### Trainer

Especificamos los parametros con los que entrenaremos nuestro modelo:

In [15]:
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de trabajo del Trainer
    num_train_epochs=3,              # Numero total de epochs sobre el que entrenaremos
    per_device_train_batch_size=16,  # Tamaño del batch de datos por cada dispositivo de entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch de datos que usaremos para evaluación
    warmup_steps=500,                # Numero de pasos que se usaran para determinar la politica de Learning Rate
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directorio de logs
    logging_steps=10,
)

Instanciamos el Trainer

In [16]:
trainer = Trainer(
    model=model,                         # modelo sobre el que haremos fine tunning
    args=training_args,                  # parametros del entrenamiento
    train_dataset=train_dataset,         # set de datos de entrenamiento
    eval_dataset=val_dataset             # set de datos de evaluación
)

In [11]:
trainer.train()

{"loss": 2.0130661964416503, "learning_rate": 1.0000000000000002e-06, "epoch": 0.06329113924050633, "step": 10}
{"loss": 2.0143117666244508, "learning_rate": 2.0000000000000003e-06, "epoch": 0.12658227848101267, "step": 20}
{"loss": 1.9196344375610352, "learning_rate": 3e-06, "epoch": 0.189873417721519, "step": 30}
{"loss": 1.9248562574386596, "learning_rate": 4.000000000000001e-06, "epoch": 0.25316455696202533, "step": 40}
{"loss": 1.8505674958229066, "learning_rate": 5e-06, "epoch": 0.31645569620253167, "step": 50}
{"loss": 1.8715176820755004, "learning_rate": 6e-06, "epoch": 0.379746835443038, "step": 60}
{"loss": 1.7870514273643494, "learning_rate": 7.000000000000001e-06, "epoch": 0.4430379746835443, "step": 70}
{"loss": 1.78349130153656, "learning_rate": 8.000000000000001e-06, "epoch": 0.5063291139240507, "step": 80}
{"loss": 1.7086456418037415, "learning_rate": 9e-06, "epoch": 0.569620253164557, "step": 90}
{"loss": 1.5932150483131409, "learning_rate": 1e-05, "epoch": 0.632911392

{"loss": 0.5704751312732697, "learning_rate": 1.6000000000000003e-05, "epoch": 1.0126582278481013, "step": 160}
{"loss": 0.5037536889314651, "learning_rate": 1.7000000000000003e-05, "epoch": 1.0759493670886076, "step": 170}
{"loss": 0.334173247218132, "learning_rate": 1.8e-05, "epoch": 1.139240506329114, "step": 180}
{"loss": 0.28362494260072707, "learning_rate": 1.9e-05, "epoch": 1.2025316455696202, "step": 190}
{"loss": 0.2360175520181656, "learning_rate": 2e-05, "epoch": 1.2658227848101267, "step": 200}
{"loss": 0.15980467796325684, "learning_rate": 2.1e-05, "epoch": 1.3291139240506329, "step": 210}
{"loss": 0.09582721889019012, "learning_rate": 2.2000000000000003e-05, "epoch": 1.3924050632911391, "step": 220}
{"loss": 0.19044460952281952, "learning_rate": 2.3000000000000003e-05, "epoch": 1.4556962025316456, "step": 230}
{"loss": 0.10874612033367156, "learning_rate": 2.4e-05, "epoch": 1.518987341772152, "step": 240}
{"loss": 0.11367769986391067, "learning_rate": 2.5e-05, "epoch": 1.

{"loss": 0.0333705710247159, "learning_rate": 3.2000000000000005e-05, "epoch": 2.0253164556962027, "step": 320}
{"loss": 0.019112420082092286, "learning_rate": 3.3e-05, "epoch": 2.088607594936709, "step": 330}
{"loss": 0.04272092580795288, "learning_rate": 3.4000000000000007e-05, "epoch": 2.151898734177215, "step": 340}
{"loss": 0.007846251130104065, "learning_rate": 3.5e-05, "epoch": 2.2151898734177213, "step": 350}
{"loss": 0.01155136078596115, "learning_rate": 3.6e-05, "epoch": 2.278481012658228, "step": 360}
{"loss": 0.049505618214607236, "learning_rate": 3.7e-05, "epoch": 2.3417721518987342, "step": 370}
{"loss": 0.043567579984664914, "learning_rate": 3.8e-05, "epoch": 2.4050632911392404, "step": 380}
{"loss": 0.029395869374275206, "learning_rate": 3.9000000000000006e-05, "epoch": 2.4683544303797467, "step": 390}
{"loss": 0.049667558073997496, "learning_rate": 4e-05, "epoch": 2.5316455696202533, "step": 400}
{"loss": 0.01619624048471451, "learning_rate": 4.1e-05, "epoch": 2.594936

TrainOutput(global_step=474, training_loss=0.5901083445023226)

In [13]:
!tensorboard --logdir ./logs --bind_all

2020-09-09 21:08:55.512290: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-09-09 21:08:55.512390: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-09-09 21:08:55.512411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorBoard 2.3.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


Verifiquemos la performance de nuestro modelo

In [18]:
predictions = trainer.predict(test_dataset=val_dataset).predictions

Para evaluar el modelo, primero deberemos obtener cual es la categoria que obtuvo la mayor probabilidad en la clasificación:

In [21]:
import numpy as np

predictions = np.argmax(predictions, axis=1)

Convertimos los IDs de las categorias a los labels correctos

In [37]:
all_labels = val_dataset.get_labels()
predictions_label = [all_labels[idx] for idx in predictions]

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions_label))

              precision    recall  f1-score   support

ALIMENTACION       1.00      0.95      0.97       110
  AUTOMOCION       0.99      1.00      1.00       148
       BANCA       0.99      0.99      0.99       198
     BEBIDAS       0.99      1.00      0.99       223
    DEPORTES       1.00      0.99      1.00       216
      RETAIL       0.98      1.00      0.99       268
       TELCO       0.99      0.99      0.99        79

    accuracy                           0.99      1242
   macro avg       0.99      0.99      0.99      1242
weighted avg       0.99      0.99      0.99      1242

